### Identify what it is we want to do  / loose requirements
* Want to know where we can go camping near Mount Hood
* Want to know how many sites are at a campground
* Want to know what sites (number of sites?) are accessible, near water, toilet y/n, and allow pets

#### This looks like:  
A dataset we can query with criteria and get a list of campsites that match, ideally with info and weblinks for further research.

### Explore data to determine if / how we can do that  
* Identify attributes in RIDB with this info
* Join with data from forest service websites

### Productionalize for scale
* generic code for getting RIDB data, configured by JSON for specific locations. Store this in a db / lookup so we can onboard new campsites without a deploy
* parallelizing to reduce runtime - api rate limits
* how often should the pipeline run?
* replace or append?
* metadata - data source, ingested_on timestamp
* ERrors - just like you want to know when something isnt as expected at a campground


### What are some camping analogies that might relate to what Im trying to convey about scalability?
* Send friends to multiple campgrounds to help find sites instead of everyone going to the same place - parallelization
* Come back to rendevous point because no cell service and compare what we found - 5 sites near water, no accesable sites, 
* Ikea camping chair?

In [119]:
import json
import pandas as pd
import itertools
import requests

import config

pd.set_option('display.max_colwidth', None)

Getting a list of facilities from RIDB
lat/long  
or state as comma delimited list of 2 char state codes  
https://ridb.recreation.gov/docs  
https://ridb.recreation.gov/ridb-access-agreement 

In [13]:
ridb_facilities_url = "https://ridb.recreation.gov/api/v1/facilities"

In [197]:
# params = {"activity_id":9, "latitude":45.4977712, "longitude":-121.8211673, "radius":15}
params = {"activity_id":9, "state":"OR"}
headers = {"accept": "application/json", "apikey": "b2b7cf12-0a8a-413b-aa75-3125b41977b8"}
response = requests.get(ridb_facilities_url, params, headers=headers)
# with open('../data/RIDB_OR_facilities.json', 'w') as f:
#     json.dump(response.text, f)
camping_json  = json.loads(response.text)

In [199]:
with open('../data/RIDB_OR_facilities.json') as f:
    res = json.load(f)
    
res

'{"RECDATA":[{"FacilityID":"265224","LegacyFacilityID":"","OrgFacilityID":"77603","ParentOrgID":"131","ParentRecAreaID":"1108","FacilityName":"Glendale - Powers Bicycle Recreation Area","FacilityDescription":"\\u003cp\\u003eThe Oregon Coast is a cycling attraction that is gaining worldwide recognition for its stunning and diverse landscapes, and the areas just inland from that majestic Highway 101 are equally impressive! The Glendale-Powers route offers the opportunity to bicycle through the ancient forests\xa0on the west side of the\xa0Rogue River-Siskiyou National Forest, along the beautiful Coquille River! Escape into a world of pristine beauty as you pass through\xa0wildflower-filled meadows, ride along clean, clear waters, and enjoy the majesty of the towering trees. Every turn of the road reveals all manner of big and small wildlife, including elk, bobcat, quail, and trout. This area provides for such amazing road biking opportunities, there is even an family-friendly annual ride

In [184]:
response.status_code

200

In [189]:
response.reason

'OK'

In [190]:
camping_json.RECDATA

AttributeError: 'dict' object has no attribute 'RECDATA'

In [181]:
camping_json['RECDATA'][0].keys()

dict_keys(['FacilityID', 'LegacyFacilityID', 'OrgFacilityID', 'ParentOrgID', 'ParentRecAreaID', 'FacilityName', 'FacilityDescription', 'FacilityTypeDescription', 'FacilityUseFeeDescription', 'FacilityDirections', 'FacilityPhone', 'FacilityEmail', 'FacilityReservationURL', 'FacilityMapURL', 'FacilityAdaAccess', 'GEOJSON', 'FacilityLongitude', 'FacilityLatitude', 'Keywords', 'StayLimit', 'Reservable', 'Enabled', 'LastUpdatedDate'])

In [160]:
df_ridb_camping = pd.DataFrame(camping_json['RECDATA'])

In [161]:
df_ridb_camping.columns

Index(['FacilityID', 'LegacyFacilityID', 'OrgFacilityID', 'ParentOrgID',
       'ParentRecAreaID', 'FacilityName', 'FacilityDescription',
       'FacilityTypeDescription', 'FacilityUseFeeDescription',
       'FacilityDirections', 'FacilityPhone', 'FacilityEmail',
       'FacilityReservationURL', 'FacilityMapURL', 'FacilityAdaAccess',
       'GEOJSON', 'FacilityLongitude', 'FacilityLatitude', 'Keywords',
       'StayLimit', 'Reservable', 'Enabled', 'LastUpdatedDate'],
      dtype='object')

In [175]:
df_ridb_camping.query("FacilityID == '251434'")

,FacilityID,LegacyFacilityID,OrgFacilityID,ParentOrgID,ParentRecAreaID,FacilityName,FacilityDescription,FacilityTypeDescription,FacilityUseFeeDescription,FacilityDirections,...,FacilityMapURL,FacilityAdaAccess,GEOJSON,FacilityLongitude,FacilityLatitude,Keywords,StayLimit,Reservable,Enabled,LastUpdatedDate
7,251434,125541,AN425541,131,1106,LOST LAKE RESORT AND CAMPGROUND,"<h2>Overview</h2>\n<p>Lost Lake Campground is couched between beautiful Lost Lake and Mount Hood. From Lost Lake Butte, the valley view is breathtaking, and from the viewpoint, guests can revel in the most majestic view of Mt. Hood, its glaciers and Lost Lake. Another one of the many attractions at Lost Lake Campground is the old growth boardwalk, an interpretive trail that meanders through one of Oregon's large old growth stands.</p>\n<h2>Recreation</h2>\n<p>Lost Lake Campground offers an abundance of activities. Campers have access to a general store, which offers: canoe, kayak, historic wooden row boats, metal fishing boats and stand-up paddleboard rentals. Whether you are a professional angler looking to catch that prized German Brown or a first-timer, there is something for everyone. Additionally, there are a number of hiking trails near the campground ranging in length from 1/4 mile to over 100 miles resulting in numerous hiking and biking opportunities. Other activities include bird watching, berry picking, swimming, waterfall finding and more.</p>\n<h2>Facilities</h2>\nLost Lake Campground is adjacent to a full service resort with a variety of campsites to accommodate all types of visitors, including group, family and single campsites. In addition to the campsites, there are cabins, yurts, lodge rooms and an extensively stocked general store on site. <br/> <br/> \n\nThere is drinking water at designated locations and pit toilets throughout. Each site has a designated tenant location, picnic table and fire ring. Firewood is available for purchase at the general store. This campground has a minimum three night stay on weekends and holidays.<h2>Natural Features</h2>\nMother Nature went the extra mile when it came to Lost Lake. The surrounding topography is rugged, mountainous and magical. The flora and fauna is abundant and easily accessible. These attributes culminate in making Lost Lake Campground one of the most popular in Oregon.\n<h2>Nearby Attractions</h2>\nNearby Wahtum Lake offers a great daily excursion or hike to one of the four surrounding waterfalls. On the way up to Lost Lake or on the way down, try stopping at one of the many fruit stands that blanket the Hood River Valley.",Campground,,"Getting There:<br /> \nLost Lake Resort &amp; Campground, Hood River County<br /> \nGPS Info:<br /> \nCoordinates: 45.489840 N 121.822578 W<br /> \nFrom Portland Oregon: Take Highway 84 for approximately 58 miles, exiting at #62. Take Country Club Road to Barrett Drive turning left on Barrett Drive to Tucker Road. Turn right on Tucker Road which eventually intersects with 281 (Dee Highway); take Dee Highway to the Lost Lake Road exit, Lost Lake Road dead-ends at Lost Lake Resort &amp; Campground.",...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.8219444, 45.4888889]}",-121.821944,45.488889,,,True,True,2021-04-04


In [176]:
df_ridb_camping.query("FacilityID == '251434'")[['FacilityLongitude', 'FacilityLatitude']]

,FacilityLongitude,FacilityLatitude
7,-121.821944,45.488889


For each facility we want to get campground information

In [80]:
# Hmm! Not all these facilities are campgrounds

In [164]:
df_ridb_camping[['FacilityID', 'FacilityName']]

,FacilityID,FacilityName
0,233329,TILLY JANE GUARD STATION
1,232834,RILEY HORSE CAMPGROUND
2,234306,EAGLE CREEK OVERLOOK GRP SITE
3,234075,WILDWOOD RECREATION SITE
4,122940,WYETH CAMPGROUND
5,10070260,Vista Ridge Trailhead
6,272093,SHERWOOD CAMPGROUND
7,251434,LOST LAKE RESORT AND CAMPGROUND
8,232836,TOLLGATE
9,232835,STILL CREEK


In [82]:
# Get campsite data for each area, if no campsites then drop --- or should we?
# Consider how we would scale this out - 
# Lost Lake example

In [192]:
f"{ridb_facilities_url}/251434/campsites".split("/")[-2]

'251434'

In [200]:
resp = requests.get(f"{ridb_facilities_url}/251434/campsites", headers=headers)

In [201]:
resp.status_code

200

In [202]:
with open('../data/RIDB/campsites/251434.json', 'w') as f:
    json.dump(resp.text, f)

In [146]:
campsites = json.loads(resp.text)

In [147]:
len(campsites['RECDATA'])

50

In [148]:
df_campsites = pd.DataFrame(campsites['RECDATA'])

In [149]:
df_campsites

,CampsiteID,FacilityID,CampsiteName,CampsiteType,TypeOfUse,Loop,CampsiteAccessible,CampsiteLongitude,CampsiteLatitude,CreatedDate,LastUpdatedDate,ATTRIBUTES,ENTITYMEDIA,PERMITTEDEQUIPMENT
0,96115,251434,A021,RV NONELECTRIC,Overnight,A Loop,False,-121.816561,45.498111,2015-04-09,2020-10-15,"[{'AttributeName': 'Driveway Grade', 'AttributeValue': 'Slight'}, {'AttributeName': 'Max Num of Vehicles', 'AttributeValue': '1'}, {'AttributeName': 'Fire Pit', 'AttributeValue': 'Y'}, {'AttributeName': 'Privacy', 'AttributeValue': 'Y'}, {'AttributeName': 'Base Number of Vehicles', 'AttributeValue': '0'}, {'AttributeName': 'Base Number of People', 'AttributeValue': '0'}, {'AttributeName': 'Grills/Fire Ring', 'AttributeValue': 'Y'}, {'AttributeName': 'Placed on Map', 'AttributeValue': '1'}, {'AttributeName': 'Picnic Table', 'AttributeValue': 'Y'}, {'AttributeName': 'Checkin Time', 'AttributeValue': '3:00 PM'}, {'AttributeName': 'Location Rating', 'AttributeValue': 'Good'}, {'AttributeName': 'Site Rating', 'AttributeValue': 'Preferred'}, {'AttributeName': 'Capacity/Size Rating', 'AttributeValue': 'Single'}, {'AttributeName': 'Checkout Time', 'AttributeValue': '10:00 AM'}, {'AttributeName': 'Max Num of People', 'AttributeValue': '5'}, {'AttributeName': 'Campfire Allowed', 'AttributeValue': 'Yes'}, {'AttributeName': 'Pets Allowed', 'AttributeValue': 'Yes'}, {'AttributeName': 'Shade', 'AttributeValue': 'Yes'}, {'AttributeName': 'Tent Pad Length', 'AttributeValue': '6'}, {'AttributeName': 'Tent Pad Width', 'AttributeValue': '6'}, {'AttributeName': 'IS EQUIPMENT MANDATORY', 'AttributeValue': 'true'}, {'AttributeName': 'Driveway Surface', 'AttributeValue': 'Paved'}, {'AttributeName': 'Driveway Length', 'AttributeValue': '93'}, {'AttributeName': 'Driveway Entry', 'AttributeValue': 'Pull-Through'}, {'AttributeName': 'Max Vehicle Length', 'AttributeValue': '81'}, {'AttributeName': 'Site Height/Overhead Clearance', 'AttributeValue': '50'}, {'AttributeName': 'Tent Pad', 'AttributeValue': 'Y'}, {'AttributeName': 'Quiet Area', 'AttributeValue': 'Y'}, {'AttributeName': 'Hike In Distance to Site', 'AttributeValue': '0'}, {'AttributeName': 'Site Access', 'AttributeValue': 'Drive-In'}]","[{'EntityMediaID': '2cf26ca1-bbdd-4619-99ff-473b34305a01', 'MediaType': 'Image', 'EntityID': '96115', 'EntityType': 'Campsite', 'Title': 'A21', 'Subtitle': '', 'Description': 'A21', 'EmbedCode': '', 'Height': 960, 'Width': 1440, 'URL': 'https://cdn.recreation.gov/public/2019/02/06/20/29/96115_91e855f7-1823-4d4a-af4f-240f46dc2d61_1440.jpg', 'Credits': 'Lost Lake Resort & Campground', 'IsPrimary': False, 'IsPreview': False, 'IsGallery': True}, {'EntityMediaID': '9c85be9b-01eb-4640-9497-06601025e16e', 'MediaType': 'Image', 'EntityID': '96115', 'EntityType': 'Campsite', 'Title': 'A21', 'Subtitle': '', 'Description': 'A21', 'EmbedCode': '', 'Height': 960, 'Width': 1440, 'URL': 'https://cdn.recreation.gov/public/2019/02/06/20/29/96115_faf7d6c8-6a94-4c37-bf7b-5a483aba4699_1440.jpg', 'Credits': 'Lost Lake Resort & Campground', 'IsPrimary': False, 'IsPreview': False, 'IsGallery': True}, {'EntityMediaID': 'b68cc09c-733a-4687-b0a0-8c9b2bfc4d92', 'MediaType': 'Image', 'EntityID': '96115', 'EntityType': 'Campsite', 'Title': 'A21', 'Subtitle': '', 'Description': 'A21', 'EmbedCode': '', 'Height': 340, 'Width': 1440, 'URL': 'https://cdn.recreation.gov/public/2019/02/06/20/28/96115_63acc8ef-d4bc-4601-9bca-650cebe96676_1440.jpg', 'Credits': 'Lost Lake Resort and Campground', 'IsPrimary': True, 'IsPreview': False, 'IsGallery': False}, {'EntityMediaID': '5a780c65-850b-4022-9da9-c61a25d03a70', 'MediaType': 'Image', 'EntityID': '96115', 'EntityType': 'Campsite', 'Title': 'A21', 'Subtitle': '', 'Description': 'A21', 'EmbedCode': '', 'Height': 526, 'Width': 700, 'URL': 'https://cdn.recreation.gov/public/2019/02/06/20/28/96115_691dff65-bf73-4186-b8c9-4a321e8cda6f_700.jpg', 'Credits': 'Lost Lake Resort and Campground', 'IsPrimary': False, 'IsPreview': True, 'IsGallery': False}, {'EntityMediaID': '567f9

50 campsites, compare with the lostlake website that says 125:  
https://www.fs.usda.gov/recarea/mthood/recarea/?recid=53228 45.48889, -121.82194

In [150]:
[entry for entry in df_campsites.iloc[0].ATTRIBUTES]

[{'AttributeName': 'Driveway Grade', 'AttributeValue': 'Slight'},
 {'AttributeName': 'Max Num of Vehicles', 'AttributeValue': '1'},
 {'AttributeName': 'Fire Pit', 'AttributeValue': 'Y'},
 {'AttributeName': 'Privacy', 'AttributeValue': 'Y'},
 {'AttributeName': 'Base Number of Vehicles', 'AttributeValue': '0'},
 {'AttributeName': 'Base Number of People', 'AttributeValue': '0'},
 {'AttributeName': 'Grills/Fire Ring', 'AttributeValue': 'Y'},
 {'AttributeName': 'Placed on Map', 'AttributeValue': '1'},
 {'AttributeName': 'Picnic Table', 'AttributeValue': 'Y'},
 {'AttributeName': 'Checkin Time', 'AttributeValue': '3:00 PM'},
 {'AttributeName': 'Location Rating', 'AttributeValue': 'Good'},
 {'AttributeName': 'Site Rating', 'AttributeValue': 'Preferred'},
 {'AttributeName': 'Capacity/Size Rating', 'AttributeValue': 'Single'},
 {'AttributeName': 'Checkout Time', 'AttributeValue': '10:00 AM'},
 {'AttributeName': 'Max Num of People', 'AttributeValue': '5'},
 {'AttributeName': 'Campfire Allowed', '

Do the campsite attributes have the information we are looking for? Near water, accessible...

In [151]:
set(itertools.chain(*df_campsites['ATTRIBUTES'].apply(lambda x: [entry['AttributeName'] for entry in x])))

{'Accessibility',
 'BBQ',
 'Base Number of People',
 'Base Number of Vehicles',
 'Campfire Allowed',
 'Capacity/Size Rating',
 'Checkin Time',
 'Checkout Time',
 'Double Driveway',
 'Driveway Entry',
 'Driveway Grade',
 'Driveway Length',
 'Driveway Surface',
 'Fire Pit',
 'Grills/Fire Ring',
 'Hike In Distance to Site',
 'Horse Hitching Post',
 'Horse Stall/Corral',
 'IS EQUIPMENT MANDATORY',
 'Location Rating',
 'Max Num of People',
 'Max Num of Vehicles',
 'Max Vehicle Length',
 'Min Num of People',
 'Pets Allowed',
 'Picnic Table',
 'Placed on Map',
 'Platform',
 'Privacy',
 'Proximity to Water',
 'Quiet Area',
 'Shade',
 'Site Access',
 'Site Height/Overhead Clearance',
 'Site Rating',
 'Tent Pad',
 'Tent Pad Length',
 'Tent Pad Width'}

near water: "Proximity to Water"   
pets allowed: "Pets Allowed"   
accessibility: "Accessibility" - boolean  
toilet?  - no info, but thats what the forest service sites have

In [152]:
df_campsites.to_json('../data/ridb_campsites_mock.json', orient='records')

In [153]:
with open('../data/ridb_campsites_mock.json') as f:
    ridb_json = json.loads(f.read())
    
df = pd.json_normalize(ridb_json, "ATTRIBUTES", ["CampsiteID", "FacilityID"])

In [154]:
df

,AttributeName,AttributeValue,CampsiteID,FacilityID
0,Driveway Grade,Slight,96115,251434
1,Max Num of Vehicles,1,96115,251434
2,Fire Pit,Y,96115,251434
3,Privacy,Y,96115,251434
4,Base Number of Vehicles,0,96115,251434
...,...,...,...,...
1371,Campfire Allowed,Yes,96085,251434
1372,Max Num of Vehicles,1,96085,251434
1373,Driveway Surface,Paved,96085,251434
1374,Driveway Grade,Slight,96085,251434


In [157]:
df.query("AttributeName == 'Pets Allowed'").head()

,AttributeName,AttributeValue,CampsiteID,FacilityID
16,Pets Allowed,Yes,96115,251434
43,Pets Allowed,Yes,96043,251434
70,Pets Allowed,Yes,96075,251434
105,Pets Allowed,Yes,96104,251434
123,Pets Allowed,"Domestic,Horse",96032,251434


In [159]:
df.query("AttributeName == 'Proximity to Water'")

,AttributeName,AttributeValue,CampsiteID,FacilityID
425,Proximity to Water,Lakefront,96099,251434
542,Proximity to Water,Lakefront,96278,251434
638,Proximity to Water,Lakefront,96224,251434
1152,Proximity to Water,Lakefront,96264,251434
1248,Proximity to Water,Lakefront,96303,251434


In [170]:
df_ridb_camping.dtypes

FacilityID                    object
LegacyFacilityID              object
OrgFacilityID                 object
ParentOrgID                   object
ParentRecAreaID               object
FacilityName                  object
FacilityDescription           object
FacilityTypeDescription       object
FacilityUseFeeDescription     object
FacilityDirections            object
FacilityPhone                 object
FacilityEmail                 object
FacilityReservationURL        object
FacilityMapURL                object
FacilityAdaAccess             object
GEOJSON                       object
FacilityLongitude            float64
FacilityLatitude             float64
Keywords                      object
StayLimit                     object
Reservable                      bool
Enabled                         bool
LastUpdatedDate               object
dtype: object

In [171]:
df.merge(df_ridb_camping, on='FacilityID', how='left')

,AttributeName,AttributeValue,CampsiteID,FacilityID,LegacyFacilityID,OrgFacilityID,ParentOrgID,ParentRecAreaID,FacilityName,FacilityDescription,...,FacilityMapURL,FacilityAdaAccess,GEOJSON,FacilityLongitude,FacilityLatitude,Keywords,StayLimit,Reservable,Enabled,LastUpdatedDate
0,Driveway Grade,Slight,96115,251434,125541,AN425541,131,1106,LOST LAKE RESORT AND CAMPGROUND,"<h2>Overview</h2>\n<p>Lost Lake Campground is couched between beautiful Lost Lake and Mount Hood. From Lost Lake Butte, the valley view is breathtaking, and from the viewpoint, guests can revel in the most majestic view of Mt. Hood, its glaciers and Lost Lake. Another one of the many attractions at Lost Lake Campground is the old growth boardwalk, an interpretive trail that meanders through one of Oregon's large old growth stands.</p>\n<h2>Recreation</h2>\n<p>Lost Lake Campground offers an abundance of activities. Campers have access to a general store, which offers: canoe, kayak, historic wooden row boats, metal fishing boats and stand-up paddleboard rentals. Whether you are a professional angler looking to catch that prized German Brown or a first-timer, there is something for everyone. Additionally, there are a number of hiking trails near the campground ranging in length from 1/4 mile to over 100 miles resulting in numerous hiking and biking opportunities. Other activities include bird watching, berry picking, swimming, waterfall finding and more.</p>\n<h2>Facilities</h2>\nLost Lake Campground is adjacent to a full service resort with a variety of campsites to accommodate all types of visitors, including group, family and single campsites. In addition to the campsites, there are cabins, yurts, lodge rooms and an extensively stocked general store on site. <br/> <br/> \n\nThere is drinking water at designated locations and pit toilets throughout. Each site has a designated tenant location, picnic table and fire ring. Firewood is available for purchase at the general store. This campground has a minimum three night stay on weekends and holidays.<h2>Natural Features</h2>\nMother Nature went the extra mile when it came to Lost Lake. The surrounding topography is rugged, mountainous and magical. The flora and fauna is abundant and easily accessible. These attributes culminate in making Lost Lake Campground one of the most popular in Oregon.\n<h2>Nearby Attractions</h2>\nNearby Wahtum Lake offers a great daily excursion or hike to one of the four surrounding waterfalls. On the way up to Lost Lake or on the way down, try stopping at one of the many fruit stands that blanket the Hood River Valley.",...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.8219444, 45.4888889]}",-121.821944,45.488889,,,True,True,2021-04-04
1,Max Num of Vehicles,1,96115,251434,125541,AN425541,131,1106,LOST LAKE RESORT AND CAMPGROUND,"<h2>Overview</h2>\n<p>Lost Lake Campground is couched between beautiful Lost Lake and Mount Hood. From Lost Lake Butte, the valley view is breathtaking, and from the viewpoint, guests can revel in the most majestic view of Mt. Hood, its glaciers and Lost Lake. Another one of the many attractions at Lost Lake Campground is the old growth boardwalk, an interpretive trail that meanders through one of Oregon's large old growth stands.</p>\n<h2>Recreation</h2>\n<p>Lost Lake Campground offers an abundance of activities. Campers have access to a general store, which offers: canoe, kayak, historic wooden row boats, metal fishing boats and stand-up paddleboard rentals. Whether you are a professional angler looking to catch that prized German Brown or a first-timer, there is something for everyone. Additionally, there are a number of hiking trails near the campground ranging in length from 1/4 mile to over 100 miles resulting in numerous hiking and biking opportunities. Other activities include bird watching, berry picking, swimming, waterfall finding and more.</p>\n<h2>Facilities</h2>\nLost Lake Campground is adjacent to a full service resort with a variety of campsites to a